# Lab 6: Multi-Agent Orchestration with Microsoft Agent Framework

Extend single agents into **coordinated multi-agent systems** using the orchestration patterns from Microsoft Agent Framework.

## Learning Objectives

By the end of this lab, you will:
1. Understand **why multi-agent systems** outperform single agents for complex tasks
2. Implement **5 orchestration patterns**: Concurrent, Sequential, Group Chat, Magentic, Handoff
3. Know **when to use** each pattern based on your use case
4. Build a **real-world multi-agent application** for research assistance

## Prerequisites

| Requirement | Setup |
|------------|-------|
| Lab 3 completed | Single agent with tool calling |
| Python environment | Run `make start_env` from project root |
| Azure CLI logged in | Run `az login` in terminal |
| Azure AI Foundry Project | [Create at ai.azure.com](https://ai.azure.com) |

## Environment Variables

Same as Lab 3 - your `.env` file should have:

```bash
AZURE_AI_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com
AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4o
```

## Why Multi-Agent Systems?

**Single agents** are powerful but have limitations:

| Limitation | Multi-Agent Solution |
|------------|----------------------|
| Context window limits | Distribute context across specialized agents |
| Single perspective | Multiple expert viewpoints on the same problem |
| Sequential bottleneck | Parallel processing with concurrent agents |
| Complex reasoning | Break down into agent collaboration |
| Quality control | Reviewer agents validate work |

### The Key Insight

> **Multi-agent systems** treat agents as team members, each with specialized skills, who collaborate to solve problems that would overwhelm a single agent.

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Single Agent vs Multi-Agent                      │
├─────────────────────────────┬───────────────────────────────────────┤
│       Single Agent          │           Multi-Agent                 │
│                             │                                       │
│   ┌─────────────────┐       │    ┌─────────┐  ┌─────────┐           │
│   │                 │       │    │Researcher│  │ Writer  │          │
│   │   One Agent     │       │    └────┬────┘  └────┬────┘           │
│   │   Does All      │       │         │            │                │
│   │                 │       │    ┌────┴────────────┴────┐           │
│   └─────────────────┘       │    │    Orchestrator      │           │
│                             │    └──────────┬───────────┘           │
│   ✗ Context limits          │               │                       │
│   ✗ Single perspective      │    ┌──────────┴───────────┐           │
│   ✗ No collaboration        │    │ Reviewer │ │ Analyst │           │
│                             │    └──────────┘ └─────────┘           │ 
│                             │                                       │
│                             │   ✓ Specialized expertise             │
│                             │   ✓ Parallel processing               │
│                             │   ✓ Collaborative refinement          │
└─────────────────────────────┴───────────────────────────────────────┘
```

## The 5 Orchestration Patterns

Microsoft Agent Framework provides **5 built-in orchestration patterns**:

| Pattern | Topology | Description | Best For |
|---------|----------|-------------|----------|
| **Concurrent** | Fan-out | All agents process the same task in parallel | Parallel analysis, ensemble reasoning |
| **Sequential** | Pipeline | Each agent passes results to the next | Step-by-step workflows, multi-stage processing |
| **Group Chat** | Star (manager) | Central manager coordinates turn-taking | Iterative refinement, collaborative problem-solving |
| **Magentic** | Star (planner) | Manager with planning for complex tasks | Complex, generalist multi-agent collaboration |
| **Handoff** | Mesh | Agents pass control dynamically without central manager | Dynamic workflows, expert escalation |

### Visual Comparison

```
┌──────────────────┐  ┌──────────────────┐  ┌──────────────────┐
│    CONCURRENT    │  │    SEQUENTIAL    │  │    GROUP CHAT    │
│                  │  │                  │  │                  │
│      Input       │  │  A ──▶ B ──▶ C   │  │    ┌────────┐    │
│        │         │  │                  │  │    │Manager │    │
│    ┌───┼───┐     │  │  Pipeline flow   │  │    └───┬────┘    │
│    ▼   ▼   ▼     │  │                  │  │   ┌────┼────┐    │
│   [A] [B] [C]    │  │                  │  │   ▼    ▼    ▼    │
│    │   │   │     │  │                  │  │  [A]  [B]  [C]   │
│    └───┼───┘     │  │                  │  │                  │
│        ▼         │  │                  │  │  Star topology   │
│     Aggregate    │  │                  │  │                  │
└──────────────────┘  └──────────────────┘  └──────────────────┘

┌──────────────────┐  ┌──────────────────┐
│     MAGENTIC     │  │      HANDOFF     │
│                  │  │                  │
│  ┌───────────┐   │  │  [A]◀──────▶[B]  │
│  │  Planner  │   │  │   │\        /│   │
│  │  Manager  │   │  │   │ \      / │   │
│  └─────┬─────┘   │  │   │  \    /  │   │
│   ┌────┼────┐    │  │   │   \  /   │   │
│   ▼    ▼    ▼    │  │   ▼    \/    ▼   │
│  [A]  [B]  [C]   │  │  [C]◀──────▶[D]  │
│                  │  │                  │
│  With planning   │  │  Mesh topology   │
│  & progress      │  │  Direct handoffs │
└──────────────────┘  └──────────────────┘
```

## Step 1: Setup and Imports

In [3]:
import os
import json
import asyncio
from pathlib import Path
from typing import Annotated, Any

from dotenv import load_dotenv

# Ensure Azure CLI is in PATH (needed for notebook kernel)
homebrew_paths = ["/opt/homebrew/bin", "/usr/local/bin"]
current_path = os.environ.get("PATH", "")
for p in homebrew_paths:
    if p not in current_path:
        os.environ["PATH"] = p + ":" + current_path
        current_path = os.environ["PATH"]

# Load environment variables
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Loaded .env from {env_path.resolve()}")
else:
    load_dotenv()
    print("⚠️  No .env found in parent directory")

# Verify required env vars
required_vars = ["AZURE_AI_PROJECT_ENDPOINT", "AZURE_AI_MODEL_DEPLOYMENT_NAME"]
missing = [v for v in required_vars if not os.getenv(v)]
if missing:
    print(f"❌ Missing environment variables: {missing}")
else:
    print("✅ Environment configured")
    print(f"   Project: {os.getenv('AZURE_AI_PROJECT_ENDPOINT')[:50]}...")
    print(f"   Model: {os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME')}")

✅ Loaded .env from /Users/pablo/Desktop/githubRepos/teaching/northwestern/northwestern-fy26-msai-foundry-agentic-ai/.env
✅ Environment configured
   Project: https://aihubnw3478841489.services.ai.azure.com/ap...
   Model: gpt-4o


## Step 2: Install Microsoft Agent Framework

Microsoft Agent Framework is a Python library for building multi-agent systems. It provides:
- **Agent abstractions** - Create agents with instructions and tools
- **Orchestration patterns** - Built-in patterns for agent coordination
- **Azure integration** - Native support for Azure OpenAI and AI Foundry

In [2]:
# Install the Microsoft Agent Framework
# Note: This package is still in preview - requires --pre flag
%pip install -q agent-framework --pre

print("\n✅ Microsoft Agent Framework installed")

/Users/pablo/Desktop/githubRepos/teaching/northwestern/northwestern-fy26-msai-foundry-agentic-ai/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.

✅ Microsoft Agent Framework installed


## Step 3: Create the Chat Client

The `AzureOpenAIChatClient` is the foundation for creating agents. It connects to your Azure OpenAI deployment.

**Key Concept**: A single chat client can create multiple agents, each with different instructions and tools.

### Environment Variables

The client reads from these environment variables automatically:
- `AZURE_OPENAI_ENDPOINT` or `AZURE_AI_PROJECT_ENDPOINT`
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME` or `AZURE_AI_MODEL_DEPLOYMENT_NAME`

Or you can pass them explicitly to the constructor.

In [4]:
from azure.identity import AzureCliCredential, get_bearer_token_provider
from agent_framework.azure import AzureOpenAIChatClient

# Initialize credential and token provider with correct scope for Azure AI
credential = AzureCliCredential()
token_provider = get_bearer_token_provider(
    credential, 
    "https://cognitiveservices.azure.com/.default"  # Required scope for Azure AI
)

# Initialize the Azure OpenAI chat client
# Note: Parameter is 'ad_token_provider', not 'azure_ad_token_provider'
chat_client = AzureOpenAIChatClient(
    ad_token_provider=token_provider,
    endpoint=os.getenv("AZURE_AI_PROJECT_ENDPOINT"),
    deployment_name=os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME"),
)

print("✅ Azure OpenAI Chat Client initialized")
print(f"   Endpoint: {os.getenv('AZURE_AI_PROJECT_ENDPOINT')[:50]}...")
print(f"   Model: {os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME')}")

✅ Azure OpenAI Chat Client initialized
   Endpoint: https://aihubnw3478841489.services.ai.azure.com/ap...
   Model: gpt-4o


# Pattern 1: Concurrent Orchestration

## When to Use

- **Parallel analysis** - Multiple experts analyze the same input
- **Ensemble reasoning** - Combine diverse perspectives
- **Independent subtasks** - Tasks don't depend on each other

## Architecture

```
                    User Input
                        │
          ┌─────────────┼─────────────┐
          │             │             │
          ▼             ▼             ▼
    ┌──────────┐  ┌──────────┐  ┌──────────┐
    │Researcher│  │ Marketer │  │  Legal   │
    │          │  │          │  │          │
    └────┬─────┘  └────┬─────┘  └────┬─────┘
         │             │             │
         └─────────────┼─────────────┘
                       │
                       ▼
               Aggregated Results
```

**How it works**:
1. Same input is broadcast to all agents
2. Agents process **in parallel** (no dependencies)
3. Results are **aggregated** at the end

In [5]:
from agent_framework import ConcurrentBuilder, ChatMessage, WorkflowOutputEvent, Role

# ============================================================
# STEP 1: Create specialized agents
# ============================================================

# Each agent has a unique perspective on the same problem
researcher = chat_client.as_agent(
    instructions=(
        "You're an expert market and product researcher. Given a prompt, provide concise, "
        "factual insights, opportunities, and risks. Focus on data-driven analysis."
    ),
    name="researcher",
)

marketer = chat_client.as_agent(
    instructions=(
        "You're a creative marketing strategist. Craft compelling value propositions "
        "and target messaging aligned to the prompt. Focus on audience and messaging."
    ),
    name="marketer",
)

legal = chat_client.as_agent(
    instructions=(
        "You're a cautious legal/compliance reviewer. Highlight constraints, disclaimers, "
        "and policy concerns. Focus on risk mitigation and compliance."
    ),
    name="legal",
)

print("✅ Created 3 specialized agents:")
print("   • researcher - Market analysis")
print("   • marketer - Marketing strategy")
print("   • legal - Compliance review")

✅ Created 3 specialized agents:
   • researcher - Market analysis
   • marketer - Marketing strategy
   • legal - Compliance review


In [6]:
# ============================================================
# STEP 2: Build the Concurrent Workflow
# ============================================================

concurrent_workflow = (
    ConcurrentBuilder()
    .participants([researcher, marketer, legal])  # All agents receive the same input
    .build()
)

print("✅ Concurrent workflow built")
print("   All 3 agents will process the input in parallel")

✅ Concurrent workflow built
   All 3 agents will process the input in parallel


In [ ]:
# ============================================================
# STEP 3: Run the Concurrent Workflow
# ============================================================

async def run_concurrent_demo():
    """Run the concurrent workflow and display results."""
    
    task = "We are launching a new AI-powered tutoring app for graduate students."
    
    print(f"📝 Task: {task}")
    print("\n" + "=" * 70)
    print("Running concurrent analysis (all agents work in parallel)...")
    print("=" * 70 + "\n")
    
    output_evt = None
    async for event in concurrent_workflow.run_stream(task):
        if isinstance(event, WorkflowOutputEvent):
            output_evt = event
    
    if output_evt:
        print("===== Aggregated Results from All Agents =====")
        messages = output_evt.data
        for i, msg in enumerate(messages, start=1):
            name = msg.author_name if msg.author_name else "user"
            print(f"\n{'-' * 60}")
            print(f"{i:02d} [{name.upper()}]")
            print(f"{'-' * 60}")
            print(msg.text[:500] + "..." if len(msg.text) > 500 else msg.text)

# Run the demo
await run_concurrent_demo()

### Key Takeaways: Concurrent Pattern

| Aspect | Details |
|--------|--------|
| **Execution** | All agents run simultaneously |
| **Dependencies** | None - agents are independent |
| **Output** | Aggregated list of all responses |
| **Best for** | Getting multiple perspectives quickly |

# Pattern 2: Sequential Orchestration

## When to Use

- **Pipeline processing** - Each step builds on the previous
- **Multi-stage workflows** - Draft → Review → Polish
- **Dependent transformations** - Output becomes next input

## Architecture

```
    User Input
        │
        ▼
  ┌──────────┐     Full context
  │  Writer  │────────────────────┐
  └────┬─────┘                    │
       │                          ▼
       │ output            ┌──────────┐
       └──────────────────▶│ Reviewer │
                           └────┬─────┘
                                │
                                ▼
                          Final Output
```

**How it works**:
1. First agent processes input
2. **Full conversation history** passes to next agent
3. Each agent builds on previous work

In [8]:
from agent_framework import SequentialBuilder

# ============================================================
# STEP 1: Create pipeline agents
# ============================================================

# Writer creates initial content
writer = chat_client.as_agent(
    instructions=(
        "You are a concise copywriter. Write a compelling marketing paragraph "
        "based on the prompt. Keep it under 100 words."
    ),
    name="writer",
)

# Reviewer provides feedback
reviewer = chat_client.as_agent(
    instructions=(
        "You are a thoughtful content reviewer. Provide brief, constructive feedback "
        "on the previous message. Suggest 2-3 specific improvements."
    ),
    name="reviewer",
)

# Editor creates final version
editor = chat_client.as_agent(
    instructions=(
        "You are a skilled editor. Based on the original content and feedback, "
        "create an improved final version. Apply the reviewer's suggestions."
    ),
    name="editor",
)

print("✅ Created 3 pipeline agents:")
print("   writer → reviewer → editor")

✅ Created 3 pipeline agents:
   writer → reviewer → editor


In [9]:
# ============================================================
# STEP 2: Build the Sequential Workflow
# ============================================================

sequential_workflow = (
    SequentialBuilder()
    .participants([writer, reviewer, editor])  # Order matters!
    .build()
)

print("✅ Sequential workflow built")
print("   Pipeline: writer → reviewer → editor")

✅ Sequential workflow built
   Pipeline: writer → reviewer → editor


In [ ]:
# ============================================================
# STEP 3: Run the Sequential Workflow
# ============================================================

async def run_sequential_demo():
    """Run the sequential workflow showing the pipeline."""
    
    task = "Write a tagline for an AI-powered research assistant for PhD students."
    
    print(f"📝 Task: {task}")
    print("\n" + "=" * 70)
    print("Running sequential pipeline (writer → reviewer → editor)...")
    print("=" * 70 + "\n")
    
    output_evt = None
    async for event in sequential_workflow.run_stream(task):
        if isinstance(event, WorkflowOutputEvent):
            output_evt = event
    
    if output_evt:
        print("===== Pipeline Results =====")
        messages = output_evt.data
        for i, msg in enumerate(messages, start=1):
            name = msg.author_name or ("assistant" if msg.role == Role.ASSISTANT else "user")
            print(f"\n{'-' * 60}")
            print(f"Step {i}: [{name.upper()}]")
            print(f"{'-' * 60}")
            print(msg.text)

# Run the demo
await run_sequential_demo()

### Key Takeaways: Sequential Pattern

| Aspect | Details |
|--------|--------|
| **Execution** | One agent at a time, in order |
| **Context** | Full conversation history passed forward |
| **Dependencies** | Each agent depends on previous output |
| **Best for** | Draft → Review → Finalize workflows |

# Pattern 3: Group Chat Orchestration

## When to Use

- **Iterative refinement** - Multiple rounds of discussion
- **Collaborative problem-solving** - Agents debate and refine
- **Controlled conversations** - Manager decides who speaks

## Architecture

```
                    User Input
                        │
                        ▼
               ┌────────────────┐
               │   Orchestrator │  ◀── Decides who speaks next
               │    (Manager)   │
               └───────┬────────┘
              ┌────────┼────────┐
              │        │        │
              ▼        ▼        ▼
        ┌─────────┐ ┌─────────┐ ┌─────────┐
        │Researcher│ │ Writer  │ │Reviewer │
        └─────────┘ └─────────┘ └─────────┘
              │        │        │
              └────────┼────────┘
                       │
                       ▼
              All see full history
```

**How it works**:
1. **Orchestrator** selects which agent speaks next
2. All agents see the **full conversation history**
3. **Iterative** - conversation continues until termination condition

In [ ]:
from agent_framework import GroupChatBuilder, GroupChatState, ChatAgent

# ============================================================
# STEP 1: Create group chat agents
# ============================================================

# Using ChatAgent directly with the chat_client
gc_researcher = ChatAgent(
    name="Researcher",
    description="Collects relevant background information.",
    instructions="Gather concise facts that help answer the question. Be brief and factual. Max 50 words.",
    chat_client=chat_client,
)

gc_writer = ChatAgent(
    name="Writer",
    description="Synthesizes polished answers using gathered information.",
    instructions="Compose clear, structured answers using the research. Be comprehensive but concise.",
    chat_client=chat_client,
)

gc_critic = ChatAgent(
    name="Critic",
    description="Reviews and improves the written content.",
    instructions="Review the Writer's output. Suggest specific improvements. Be constructive.",
    chat_client=chat_client,
)

print("✅ Created 3 group chat agents:")
print("   • Researcher - Gathers facts")
print("   • Writer - Creates content")
print("   • Critic - Reviews and improves")

In [ ]:
# ============================================================
# STEP 2: Define Speaker Selection Logic
# ============================================================

def round_robin_selector(state: GroupChatState) -> str:
    """
    Simple round-robin speaker selection.
    Rotates through participants based on the current round.
    """
    participant_names = list(state.participants.keys())
    selected = participant_names[state.current_round % len(participant_names)]
    print(f"   🎯 Selector chose: {selected} (round {state.current_round})")
    return selected

print("✅ Defined round-robin speaker selector")

In [ ]:
# ============================================================
# STEP 3: Build the Group Chat Workflow
# ============================================================

group_chat_workflow = (
    GroupChatBuilder()
    .with_select_speaker_func(round_robin_selector)
    .participants([gc_researcher, gc_writer, gc_critic])
    # Terminate after 4 turns: Researcher → Writer → Critic → Writer
    .with_termination_condition(lambda conversation: len(conversation) >= 5)
    .build()
)

print("✅ Group chat workflow built")
print("   Termination: After 4 agent turns")

In [ ]:
# ============================================================
# STEP 4: Run the Group Chat
# ============================================================

from agent_framework import AgentResponseUpdateEvent

async def run_group_chat_demo():
    """Run group chat with streaming output."""
    
    task = "What are the key benefits of using AI agents in education?"
    
    print(f"📝 Task: {task}")
    print("\n" + "=" * 70)
    print("Running group chat (manager selects speakers)...")
    print("=" * 70 + "\n")
    
    last_executor_id = None
    output_event = None
    
    async for event in group_chat_workflow.run_stream(task):
        if isinstance(event, AgentResponseUpdateEvent):
            eid = event.executor_id
            if eid != last_executor_id:
                if last_executor_id is not None:
                    print("\n")
                print(f"[{eid}]: ", end="", flush=True)
                last_executor_id = eid
            print(event.data, end="", flush=True)
        elif isinstance(event, WorkflowOutputEvent):
            output_event = event
    
    print("\n\n" + "=" * 70)
    print("Group chat completed!")
    print("=" * 70)

# Run the demo
await run_group_chat_demo()

### Key Takeaways: Group Chat Pattern

| Aspect | Details |
|--------|--------|
| **Topology** | Star - manager in the center |
| **Speaker Selection** | Manager decides (round-robin, AI-based, or custom) |
| **Context** | All agents see full conversation |
| **Best for** | Iterative refinement, debate, collaborative writing |

# Pattern 4: Magentic Orchestration

## When to Use

- **Complex, open-ended tasks** - Solution path unknown in advance
- **Dynamic planning** - Manager creates and adapts plans
- **Generalist multi-agent** - Inspired by MagenticOne paper

## Architecture

```
                    User Input
                        │
                        ▼
            ┌───────────────────────┐
            │    Magentic Manager   │
            │  ┌─────────────────┐  │
            │  │   Planner       │  │  ◀── Creates & tracks plan
            │  │   Progress      │  │  ◀── Detects stalls
            │  │   Tracker       │  │  ◀── Adapts dynamically
            │  └─────────────────┘  │
            └───────────┬───────────┘
               ┌────────┼────────┐
               │        │        │
               ▼        ▼        ▼
         ┌─────────┐ ┌─────────┐ ┌─────────┐
         │Researcher│ │ Coder   │ │ Analyst │
         └─────────┘ └─────────┘ └─────────┘
```

**How it works**:
1. Manager **creates a plan** for the task
2. Dynamically selects agents based on **current progress**
3. **Tracks progress** and detects stalls
4. Can **re-plan** if the approach isn't working

In [ ]:
from agent_framework import MagenticBuilder, ChatAgent

# ============================================================
# STEP 1: Create specialized agents for Magentic
# ============================================================

# Researcher for information gathering
mag_researcher = ChatAgent(
    name="ResearcherAgent",
    description="Specialist in research and information gathering.",
    instructions=(
        "You are a Researcher. Find and summarize relevant information. "
        "Be concise and factual. Cite sources when possible."
    ),
    chat_client=chat_client,
)

# Analyst for data analysis
mag_analyst = ChatAgent(
    name="AnalystAgent",
    description="Specialist in data analysis and quantitative reasoning.",
    instructions=(
        "You are an Analyst. Process data, perform calculations, and provide "
        "quantitative insights. Show your reasoning."
    ),
    chat_client=chat_client,
)

# Synthesizer for final output
mag_synthesizer = ChatAgent(
    name="SynthesizerAgent",
    description="Specialist in synthesizing information into clear summaries.",
    instructions=(
        "You are a Synthesizer. Combine research and analysis into clear, "
        "actionable summaries. Be comprehensive yet concise."
    ),
    chat_client=chat_client,
)

# Manager agent for orchestration
mag_manager = ChatAgent(
    name="MagenticManager",
    description="Orchestrator that coordinates research and analysis workflow.",
    instructions="You coordinate a team to complete complex tasks efficiently.",
    chat_client=chat_client,
)

print("✅ Created Magentic agents:")
print("   • ResearcherAgent - Information gathering")
print("   • AnalystAgent - Data analysis")
print("   • SynthesizerAgent - Summary generation")
print("   • MagenticManager - Plan and coordinate")

In [ ]:
# ============================================================
# STEP 2: Build the Magentic Workflow
# ============================================================

magentic_workflow = (
    MagenticBuilder()
    .participants([mag_researcher, mag_analyst, mag_synthesizer])
    .with_standard_manager(
        agent=mag_manager,
        max_round_count=6,     # Maximum turns
        max_stall_count=2,     # Stalls before re-planning
        max_reset_count=1,     # Maximum re-plans
    )
    .build()
)

print("✅ Magentic workflow built")
print("   Max rounds: 6")
print("   Max stalls: 2")
print("   Max resets: 1")

In [ ]:
# ============================================================
# STEP 3: Run the Magentic Workflow
# ============================================================

from agent_framework import (
    MagenticAgentDeltaEvent,
    MagenticAgentMessageEvent,
    MagenticFinalResultEvent,
    MagenticOrchestratorMessageEvent,
)

async def run_magentic_demo():
    """Run Magentic workflow with progress tracking."""
    
    task = (
        "Compare the pros and cons of Python vs JavaScript for building AI agents. "
        "Consider: ease of use, library ecosystem, performance, and deployment options."
    )
    
    print(f"📝 Task: {task}")
    print("\n" + "=" * 70)
    print("Running Magentic orchestration (with planning)...")
    print("=" * 70 + "\n")
    
    output_event = None
    current_agent = None
    
    async for event in magentic_workflow.run_stream(task):
        if isinstance(event, MagenticAgentDeltaEvent):
            # Streaming content from an agent
            if event.executor_id != current_agent:
                if current_agent is not None:
                    print("\n")
                print(f"🤖 [{event.executor_id}]: ", end="", flush=True)
                current_agent = event.executor_id
            print(event.data, end="", flush=True)
            
        elif isinstance(event, MagenticOrchestratorMessageEvent):
            print(f"\n\n📋 [Orchestrator]: {event.data[:200]}...")
                
        elif isinstance(event, MagenticFinalResultEvent):
            output_event = event
    
    if output_event:
        print("\n\n" + "=" * 70)
        print("Final Output:")
        print("=" * 70)
        print(output_event.data)

# Run the demo
await run_magentic_demo()

### Key Takeaways: Magentic Pattern

| Aspect | Details |
|--------|--------|
| **Planning** | Manager creates and tracks execution plan |
| **Progress Tracking** | Detects stalls and adapts |
| **Dynamic Selection** | Chooses agents based on task needs |
| **Best for** | Complex, multi-step tasks with unknown solution paths |

# Pattern 5: Handoff Orchestration

## When to Use

- **Customer support** - Route to appropriate specialist
- **Expert escalation** - Pass to higher-level agent when needed
- **Dynamic workflows** - Agents decide handoffs based on context

## Architecture

```
                 User Input
                     │
                     ▼
              ┌──────────────┐
              │   Triage     │  ◀── Entry point
              │   Agent      │
              └──────┬───────┘
           ┌─────────┼─────────┐
           │         │         │
           ▼         ▼         ▼
     ┌─────────┐ ┌─────────┐ ┌─────────┐
     │ Refund  │◀│  Order  │▶│ Return  │
     │ Agent   │─│  Agent  │─│ Agent   │
     └─────────┘ └─────────┘ └─────────┘
           │         │         │
           └─────────┼─────────┘
                     │
              Mesh topology:
              Agents can hand off
              directly to each other
```

**How it works**:
1. **Triage agent** receives user input
2. Agents can **handoff** control to specialists
3. **No central manager** - mesh topology
4. Interactive - may request user input

In [ ]:
from agent_framework import HandoffBuilder, ai_function
from typing import Annotated

# ============================================================
# STEP 1: Define tools for the agents
# ============================================================

@ai_function
def check_order_status(order_number: Annotated[str, "Order number to check"]) -> str:
    """Check the status of a customer order."""
    # Simulated response
    return f"Order {order_number} is currently being processed and will ship in 2 business days."

@ai_function
def process_return(order_number: Annotated[str, "Order number for return"]) -> str:
    """Initiate a return for an order."""
    return f"Return initiated for order {order_number}. Return label sent to customer's email."

@ai_function
def process_refund(order_number: Annotated[str, "Order number for refund"]) -> str:
    """Process a refund for an order."""
    return f"Refund of $99.99 processed for order {order_number}. Will appear in 3-5 business days."

print("✅ Defined 3 customer service tools")

In [ ]:
# ============================================================
# STEP 2: Create specialized agents using the shared chat client
# ============================================================
from agent_framework import ChatAgent

triage_agent = ChatAgent(
    name="triage_agent",
    instructions="You are a customer service triage agent. Analyze customer requests and determine if they need order status, returns, or refunds.",
    chat_client=chat_client
)

order_agent = ChatAgent(
    name="order_agent",
    instructions="You handle order status inquiries. Use tools to check order status.",
    chat_client=chat_client,
    tools=[check_order_status]
)

returns_agent = ChatAgent(
    name="returns_agent",
    instructions="You handle product returns. Use tools to process return requests.",
    chat_client=chat_client,
    tools=[process_return]
)

refunds_agent = ChatAgent(
    name="refunds_agent",
    instructions="You process refunds. Use tools to issue refunds.",
    chat_client=chat_client,
    tools=[process_refund]
)

print("✅ Created 4 specialized agents for handoff workflow")

In [ ]:
# ============================================================
# STEP 3: Build the Handoff Workflow
# ============================================================

handoff_workflow = (
    HandoffBuilder(
        name="customer_support_handoff",
        participants=[triage_agent, order_agent, return_agent, refund_agent],
    )
    .with_start_agent(triage_agent)  # Triage receives initial input
    .with_termination_condition(
        # End when agent provides closing message
        lambda conversation: len(conversation) > 0 and any(
            word in conversation[-1].text.lower() 
            for word in ["welcome", "thank you", "goodbye", "anything else"]
        )
    )
    # Enable autonomous mode so agents don't wait for user input
    .with_autonomous_mode()
    .build()
)

print("✅ Handoff workflow built")
print("   Start agent: triage_agent")
print("   Mode: Autonomous (agents don't wait for user input)")

In [ ]:
# ============================================================
# STEP 4: Run the handoff workflow
# ============================================================
import asyncio
from agent_framework import HandoffMessageEvent, HandoffFinalResultEvent

async def run_handoff_workflow():
    """Run the handoff workflow and observe agent transitions."""
    
    customer_request = "I ordered product ABC123 last week but it arrived damaged. I want to return it and get a refund."
    
    print("=" * 60)
    print("Customer Request:", customer_request)
    print("=" * 60)
    print("\n🔄 Processing through handoff workflow...\n")
    
    async for event in handoff_workflow.stream(customer_request):
        if isinstance(event, HandoffMessageEvent):
            agent_name = getattr(event, 'agent_name', getattr(event, 'source', 'unknown'))
            print(f"\n📢 [{agent_name}]:")
            print(f"   {event.delta if hasattr(event, 'delta') else event}")
        elif isinstance(event, HandoffFinalResultEvent):
            print(f"\n{'='*60}")
            print("🏁 Final Result:")
            print(f"   {event.value if hasattr(event, 'value') else event}")

# Run the workflow
await run_handoff_workflow()

### Key Takeaways: Handoff Pattern

| Aspect | Details |
|--------|--------|
| **Topology** | Mesh - no central manager |
| **Control Flow** | Agents hand off based on context |
| **Context** | Full history shared across all agents |
| **Best for** | Customer support, expert routing, escalation |

# Choosing the Right Pattern

## Decision Guide

```
                            START
                              │
                              ▼
                    ┌─────────────────┐
                    │ Do agents need  │
                    │ to see each     │
                    │ other's work?   │
                    └────────┬────────┘
                    NO       │        YES
               ┌─────────────┴─────────────┐
               ▼                           ▼
        ┌────────────┐              ┌────────────────┐
        │ CONCURRENT │              │ Need central   │
        │            │              │ coordinator?   │
        └────────────┘              └───────┬────────┘
                                    NO      │      YES
                               ┌───────────┴──────────┐
                               ▼                      ▼
                        ┌────────────┐         ┌────────────────┐
                        │  HANDOFF   │         │ Simple pipeline│
                        │            │         │ or iterative?  │
                        └────────────┘         └───────┬────────┘
                                              PIPELINE │ ITERATIVE
                                           ┌───────────┴───────────┐
                                           ▼                       ▼
                                    ┌────────────┐          ┌────────────────┐
                                    │ SEQUENTIAL │          │ Need planning? │
                                    │            │          └───────┬────────┘
                                    └────────────┘          NO      │     YES
                                                       ┌───────────┴──────────┐
                                                       ▼                      ▼
                                                ┌────────────┐         ┌────────────┐
                                                │ GROUP CHAT │         │  MAGENTIC  │
                                                └────────────┘         └────────────┘
```

## Pattern Summary

| Use Case | Pattern | Why |
|----------|---------|-----|
| Multiple perspectives on same input | **Concurrent** | Parallel, independent analysis |
| Draft → Review → Finalize | **Sequential** | Each step builds on previous |
| Collaborative refinement | **Group Chat** | Iterative, manager-controlled |
| Complex, unknown solution path | **Magentic** | Planning + progress tracking |
| Customer support routing | **Handoff** | Dynamic, context-based handoffs |

# Exercise: Build Your Own Multi-Agent System

Now it's your turn! Create a multi-agent system for **graduate research assistance** using the patterns you learned.

## Scenario

Build a system that helps graduate students with:
1. **Literature review** - Finding and summarizing papers
2. **Methodology critique** - Reviewing research methods
3. **Writing feedback** - Improving academic writing

## Your Task

1. Decide which orchestration pattern fits best
2. Create specialized agents
3. Build and test the workflow

Use the cell below to implement your solution:

In [ ]:
# ============================================================
# YOUR CODE HERE: Build a Research Assistant Multi-Agent System
# ============================================================

# Step 1: Create your specialized agents
# Hint: Consider agents for literature review, methodology, writing

# literature_agent = chat_client.as_agent(...)
# methodology_agent = chat_client.as_agent(...)
# writing_agent = chat_client.as_agent(...)

# Step 2: Choose and build your orchestration pattern
# Hint: Think about whether agents need to collaborate or work independently

# research_workflow = (
#     YourChosenBuilder()
#     .participants([...])
#     ...
#     .build()
# )

# Step 3: Test your workflow
# async def test_research_assistant():
#     task = "Review my research proposal on using LLMs for automated code review."
#     async for event in research_workflow.run_stream(task):
#         ...

print("🎯 Your turn! Implement your multi-agent research assistant above.")

# Summary

## What You Learned

| Topic | Key Insight |
|-------|-------------|
| **Why Multi-Agent** | Specialized agents > one generalist for complex tasks |
| **Concurrent** | Fan-out for parallel, independent analysis |
| **Sequential** | Pipeline for step-by-step workflows |
| **Group Chat** | Star topology with manager-controlled turns |
| **Magentic** | Planning + progress tracking for complex tasks |
| **Handoff** | Mesh topology for dynamic routing |

## Key Concepts

```
┌─────────────────────────────────────────────────────────────────────┐
│                Microsoft Agent Framework Patterns                   │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  • ConcurrentBuilder  → Parallel processing, aggregated results    │
│  • SequentialBuilder  → Pipeline processing, shared context        │
│  • GroupChatBuilder   → Manager-controlled collaboration           │
│  • MagenticBuilder    → Planning + dynamic agent selection         │
│  • HandoffBuilder     → Direct agent-to-agent routing              │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

## Next Steps

1. **Experiment** with different patterns for the same task
2. **Add tools** to your agents (from Labs 1-4)
3. **Implement HITL** (Human-in-the-Loop) for sensitive operations
4. **Deploy** your multi-agent system to Azure

## Resources

- [Microsoft Agent Framework Documentation](https://learn.microsoft.com/azure/ai-services/agents/)
- [MagenticOne Paper](https://arxiv.org/abs/2411.04468)
- [Multi-Agent Design Patterns](https://learn.microsoft.com/azure/ai-services/agents/concepts/multi-agent)